In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import artm
import re
import pickle

import os
import glob

from collections import Counter

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
print(artm.version())

0.9.0


## Загрузка данных в коллекцию

In [10]:
batch_vectorizer = None
if len(glob.glob(os.path.join('kos', '*.batch'))) < 1:
    print('here')
    batch_vectorizer = artm.BatchVectorizer(data_path='kos', 
                                            data_format='bow_uci', 
                                            collection_name='kos', 
                                            target_folder='kos')
else:
    batch_vectorizer = artm.BatchVectorizer(data_path='kos', 
                                            data_format='batches')

here


In [13]:
dictionary = artm.Dictionary()

if not os.path.isfile('kos/dictionary.dict'):
    dictionary.gather(data_path=batch_vectorizer.data_path)
    dictionary.save(dictionary_path='kos/dictionary.dict')

dictionary.load(dictionary_path='kos/dictionary.dict')
dictionary.load(dictionary_path='kos/dictionary.dict')

In [16]:
dictionary = batch_vectorizer.dictionary

In [30]:
num_topics = 10
num_iter = 10
class_ids = {'@default_class': 1.0}

In [48]:
model = artm.ARTM(num_topics=num_topics,
                      dictionary=dictionary,
                      cache_theta=True,
                      reuse_theta=True,
                      theta_columns_naming='title',
                      theta_name='theta',
                      class_ids=class_ids)
    # scores without class_ids
model.scores.add(artm.PerplexityScore(name='PerplexityScore', dictionary=dictionary))
model.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore',
                                             topic_names=model.topic_names))

## Прогон модели

In [49]:
model.fit_offline(batch_vectorizer, num_collection_passes=num_iter)

In [50]:
phi = model.get_phi()
theta = model.get_theta()

In [51]:
phi.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
predebate,1.559367e-05,4.771524e-10,0.000018,0.000003,0.000000e+00,8.889098e-05,0.000000e+00,8.481417e-13,5.111464e-15,1.340616e-09
barbour,8.666757e-12,0.000000e+00,0.000000,0.000007,5.805264e-07,2.536440e-04,1.983245e-05,1.508367e-12,1.180926e-12,1.898581e-07
bumblebums,0.000000e+00,9.774648e-14,0.000261,0.000000,4.716354e-04,0.000000e+00,0.000000e+00,0.000000e+00,4.017246e-13,0.000000e+00
spindizzy,5.515496e-16,6.895757e-14,0.000273,0.000000,4.903445e-04,0.000000e+00,0.000000e+00,0.000000e+00,2.716285e-14,0.000000e+00
mcentee,0.000000e+00,0.000000e+00,0.000000,0.000000,2.026940e-13,3.915129e-14,1.446391e-16,0.000000e+00,5.604493e-04,1.046842e-15


In [52]:
theta.head()

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
topic_0,0.317986,0.000000,0.296328,7.932479e-02,6.957777e-03,0.000000e+00,1.203053e-01,0.000000,6.219731e-14,2.655639e-14,...,0.405047,0.022242,8.101609e-02,2.016652e-16,8.619546e-04,2.586948e-08,0.000000e+00,1.947319e-11,2.785038e-10,0.000000
topic_1,0.357562,0.000000,0.027244,1.060059e-13,4.282160e-04,0.000000e+00,3.307888e-02,0.000000,0.000000e+00,0.000000e+00,...,0.000310,0.000011,6.015854e-07,0.000000e+00,0.000000e+00,3.074380e-01,4.563063e-16,9.136071e-03,0.000000e+00,0.000000
topic_2,0.000000,0.032342,0.000000,1.318244e-10,9.497471e-03,1.000000e+00,1.666639e-07,0.000000,0.000000e+00,1.506624e-08,...,0.000000,0.000000,0.000000e+00,8.237324e-01,3.299588e-14,1.418151e-02,6.701723e-15,8.544581e-03,0.000000e+00,0.000000
topic_3,0.000943,0.000000,0.004460,2.047598e-01,1.767253e-01,1.004865e-12,9.707098e-04,0.083137,2.864749e-02,4.849271e-03,...,0.242819,0.014091,1.657027e-02,8.908503e-09,3.865769e-01,1.513948e-01,1.362417e-01,1.340713e-04,2.210319e-01,0.057991
topic_4,0.219647,0.000946,0.519475,5.952622e-04,4.606396e-13,1.340456e-08,3.212908e-02,0.000000,1.567544e-15,3.891030e-06,...,0.001198,0.173389,3.120962e-07,1.184843e-01,3.170368e-02,0.000000e+00,0.000000e+00,1.324158e-10,0.000000e+00,0.000000


In [53]:
phi.shape, theta.shape

((6906, 10), (10, 3430))

## Подмена Фи

In [54]:
def init_custom_phi(model, class_id, class_id_phi):
    tm_info, phi_ref = model.master.attach_model(model=model.model_pwt)

    fields = tm_info.ListFields()
    token_order = np.array(fields[3][1])
    topic_order = np.array(fields[2][1])
    class_id_order = np.array(fields[5][1])

    new_phi = pd.DataFrame(data=phi_ref,
                           index=token_order,
                           columns=topic_order)

    mask = class_id_order == class_id
    current_phi = new_phi.iloc[mask].copy()
    current_phi.update(class_id_phi)
    new_phi.iloc[mask] = current_phi

    np.copyto(phi_ref, new_phi.values)
    return model

In [55]:
phi_new = pd.DataFrame(data=np.random.random(phi.shape),
                       columns=phi.columns,
                       index=phi.index)

In [56]:
phi_new.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
predebate,0.600288,0.113186,0.164752,0.391664,0.205864,0.209960,0.128473,0.184792,0.279693,0.281616
barbour,0.873536,0.535326,0.359308,0.032112,0.136268,0.640242,0.014927,0.012221,0.672071,0.148203
bumblebums,0.961663,0.108372,0.098168,0.092543,0.547058,0.109496,0.692209,0.381485,0.256309,0.742678
spindizzy,0.513241,0.552035,0.614153,0.692317,0.204203,0.980742,0.163537,0.718826,0.972189,0.934545
mcentee,0.730330,0.800260,0.706824,0.383587,0.228841,0.743540,0.562666,0.277414,0.359004,0.298519


In [57]:
model_new = init_custom_phi(model, class_id='@default_class', class_id_phi=phi_new)

In [58]:
model_new.get_phi().head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
predebate,0.600288,0.113186,0.164752,0.391664,0.205864,0.209960,0.128473,0.184792,0.279693,0.281616
barbour,0.873536,0.535326,0.359308,0.032112,0.136268,0.640242,0.014927,0.012221,0.672071,0.148203
bumblebums,0.961663,0.108372,0.098168,0.092543,0.547058,0.109496,0.692209,0.381485,0.256309,0.742678
spindizzy,0.513241,0.552035,0.614153,0.692317,0.204203,0.980742,0.163537,0.718826,0.972189,0.934545
mcentee,0.730330,0.800260,0.706824,0.383587,0.228841,0.743540,0.562666,0.277414,0.359004,0.298519


In [59]:
model_new.fit_offline(batch_vectorizer, num_collection_passes=num_iter)

In [60]:
model_new.get_phi().head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
predebate,5.698420e-05,3.413876e-07,0.000018,1.110880e-05,0.000000e+00,4.518716e-06,0.000000,5.516832e-06,2.570949e-05,1.198883e-06
barbour,3.041260e-11,0.000000e+00,0.000000,1.600570e-07,2.052859e-09,3.810915e-04,0.000001,2.186010e-15,3.417976e-14,5.736089e-10
bumblebums,1.114285e-15,1.264632e-15,0.000406,0.000000e+00,1.781738e-06,2.331677e-13,0.000000,0.000000e+00,7.513892e-15,0.000000e+00
spindizzy,2.843821e-16,1.028806e-15,0.000424,0.000000e+00,4.995859e-07,1.880039e-13,0.000000,0.000000e+00,3.983614e-15,0.000000e+00
mcentee,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.437497e-10,0.000000,0.000000e+00,4.914535e-04,0.000000e+00


## Подмена Фи

In [115]:
def init_custom_theta(model, class_id_phi):
    tm_info, phi_ref = model.master.attach_model(model=model.theta_name)

    fields = tm_info.ListFields()
    doc_order = np.array(fields[3][1])
    topic_order = np.array(fields[2][1])
    
    print(phi_ref.shape)

    new_phi = pd.DataFrame(data=phi_ref.T,
                           index=topic_order,
                           columns=doc_order)

    print(new_phi.shape)
    current_phi = new_phi.copy()
    current_phi.update(class_id_phi)
    new_phi = current_phi.copy()

    np.copyto(phi_ref, new_phi.values.T)
    return model

In [116]:
theta_new = pd.DataFrame(data=np.random.random(theta.shape),
                       columns=theta.columns,
                       index=theta.index)

In [117]:
theta_new.head()

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
topic_0,0.253828,0.105636,0.522545,0.900549,0.711446,0.157620,0.954709,0.401875,0.148614,0.301425,...,0.952692,0.367139,0.990485,0.717603,0.442356,0.144267,0.915164,0.759758,0.424313,0.298619
topic_1,0.375843,0.814147,0.653294,0.862777,0.355880,0.215285,0.305289,0.526001,0.341793,0.818205,...,0.895196,0.388949,0.743757,0.782340,0.706907,0.342647,0.122608,0.434243,0.665976,0.880371
topic_2,0.505500,0.592421,0.090751,0.490024,0.341837,0.669631,0.934827,0.792667,0.761241,0.081708,...,0.327735,0.253790,0.106632,0.661703,0.710091,0.619764,0.626211,0.389180,0.242926,0.268956
topic_3,0.500565,0.540554,0.551953,0.307033,0.699627,0.347081,0.786685,0.135277,0.062950,0.301437,...,0.287431,0.529197,0.302734,0.002980,0.979638,0.948853,0.182953,0.841014,0.900338,0.808614
topic_4,0.925963,0.738398,0.649108,0.215154,0.046559,0.930696,0.019680,0.020679,0.615051,0.672985,...,0.555221,0.977524,0.459417,0.499594,0.009958,0.845913,0.861079,0.675615,0.132110,0.564682


In [118]:
theta_new.shape

(10, 3430)

In [119]:
model_new2 = init_custom_theta(model, class_id_phi=theta_new)

(3430, 10)
(10, 3430)


In [120]:
model_new2.get_phi().head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
predebate,3.503246e-44,3.413876e-07,6.854389e-25,1.401298e-45,1.819909e-37,1.401298e-45,1.152318e-24,1.401298e-45,3.012792e-43,1.961818e-44
barbour,5.616404e-42,5.571563e-42,0.000000e+00,0.000000e+00,1.180216e-24,1.401298e-45,NaN,NaN,0.000000e+00,0.000000e+00
bumblebums,1.821259e-37,1.401298e-45,1.261169e-44,0.000000e+00,6.854282e-25,1.401298e-45,1.166505e-24,1.401298e-45,1.166704e-24,1.401298e-45
spindizzy,1.166682e-24,1.401298e-45,1.166693e-24,1.401298e-45,1.166698e-24,1.401298e-45,1.166687e-24,1.401298e-45,1.166665e-24,1.401298e-45
mcentee,1.166671e-24,1.401298e-45,1.722129e-25,1.401298e-45,1.549565e-25,1.401298e-45,NaN,NaN,1.401298e-45,0.000000e+00


In [121]:
model_new2.get_theta().head()

,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
topic_0,0.253828,0.105636,0.522545,0.900549,0.711446,0.157620,0.954709,0.401875,0.148614,0.301425,...,0.952692,0.367139,0.990485,0.717603,0.442356,0.144267,0.915164,0.759758,0.424313,0.298618
topic_1,0.375843,0.814147,0.653294,0.862777,0.355880,0.215285,0.305289,0.526001,0.341793,0.818205,...,0.895196,0.388949,0.743757,0.782340,0.706907,0.342647,0.122608,0.434243,0.665976,0.880371
topic_2,0.505500,0.592421,0.090751,0.490024,0.341837,0.669631,0.934827,0.792667,0.761241,0.081708,...,0.327735,0.253790,0.106632,0.661703,0.710091,0.619764,0.626211,0.389180,0.242926,0.268956
topic_3,0.500565,0.540554,0.551953,0.307033,0.699627,0.347081,0.786685,0.135277,0.062950,0.301437,...,0.287431,0.529197,0.302734,0.002980,0.979638,0.948853,0.182953,0.841014,0.900338,0.808614
topic_4,0.925963,0.738398,0.649108,0.215154,0.046559,0.930696,0.019680,0.020679,0.615051,0.672985,...,0.555221,0.977524,0.459417,0.499594,0.009958,0.845913,0.861079,0.675615,0.132110,0.564682
